In [1]:
#!pip install torch_sparse

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import scipy.sparse
import sklearn.linear_model

"""Define graph forming functions"""
# Import packages

import math
import joblib
import itertools
import numpy as np
import pandas as pd
from datetime import date
from tqdm import tqdm
from functools import partial
#from google.colab import files

from sklearn import tree
import sklearn.svm as svm
from sklearn import ensemble
import sklearn.metrics as skm
import sklearn.preprocessing as pp
import sklearn.linear_model as lms
import sklearn.neural_network as skl_nn
import sklearn.neighbors as neighbors
from sklearn.naive_bayes import GaussianNB
import sklearn.model_selection as model_sel

import torch

#import torch_sparse
import torch.nn as nn
# from torch_geometric.data import Data
# from torch_geometric.nn import GCNConv

In [3]:
# Input data
random_state = 2
np.random.seed(random_state)
data_path = "/content/drive/MyDrive/PhD - Climate Change Mitigation/data/GRL_for_IE/"
#'C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\models\datasets/'

max_test_set = 100000

In [4]:
def filter_for_start_yr(df, start_col, end_col) -> pd.DataFrame:
    """Convert dataframe of plants with entry for each year into dataframe with row for each year"""
    # Get rid of emissions for years before start year
    df['Age'] = df['Year'].astype(int) - df[start_col].astype(int)
    df = df[df['Age'] >= 0]

    # Get rid of emissions for years after end years
    df['ToGo'] = df[end_col].astype(int) - df['Year'].astype(int)
    df = df[df['ToGo'] >= 0]

    df[['START_YR', 'END_YR']] = df[['START_YR', 'END_YR']].astype(int)

    return df.drop(columns=['ToGo'])

def pivot_data_dfs(df:pd.DataFrame, time_col) -> [pd.DataFrame, pd.DataFrame]:
    """Pivot data dataframes to get each entry and all year/month values in rows"""
    feature_cols = [i for i in df.columns if i not in [time_col, 'Emissions']]
    df_pivoted = df.pivot(index=feature_cols,
                          columns=time_col,
                          values = 'Emissions').reset_index()
    return df_pivoted, feature_cols


def melt_data_dfs(df:pd.DataFrame, feature_cols, time_col) -> pd.DataFrame:
    """Melt data dataframes to get row per date entry for each facility"""
    melted = df.melt(id_vars=feature_cols, var_name=time_col, value_name='Emissions').dropna(subset=['Emissions'])
    melted[time_col] = melted[time_col].astype(int)
    
    if 'START_YR' in df.columns and 'END_YR' in df.columns:
        melted = filter_for_start_yr(melted, 'START_YR', 'END_YR')

    return melted

## Convert train and test sets into ML ready sets
def series_to_bins(series:pd.Series, bins:list=None, labels:list=None, positive:bool=True):
    # Convert a continuous pandas dataframe column into discrete bins
    if bins is None:
        bin_series = series[series!=0] if positive else series
        bins = [min(bin_series.min(),0)-0.01, bin_series.quantile(0.25), bin_series.quantile(0.5), bin_series.quantile(0.75), bin_series.max()+0.01]
    if labels is None: labels = list(range(len(bins)-1))

    transformer = pp.FunctionTransformer(
        pd.cut, kw_args={'bins': bins, 'labels': labels, 'retbins': False}
    )
    return bins, transformer.fit_transform(series)


def preprocess_yearly(train_set, test_set, y_col='Emissions'):
    """Digitise train and test sets"""

    # Create Y
    bins, y_train_clf = series_to_bins(train_set[y_col])
    _, y_test_clf = series_to_bins(test_set[y_col], bins=[test_set[y_col].min()-0.01]+bins[1:-1]+[test_set[y_col].max()+0.01])

    y_train_reg, y_test_reg = train_set[y_col], test_set[y_col]
    train_set, test_set = train_set.drop(columns=[y_col]), test_set.drop(columns=[y_col])

    # Create X
    # Deal with string columns
    x_enc = pp.OrdinalEncoder()
    string_cols = list(train_set.select_dtypes(include='object').columns)
    train_set[string_cols] = train_set[string_cols].astype(str)
    test_set[string_cols] = test_set[string_cols].astype(str)
    x_strings = pd.concat((train_set[string_cols], test_set[string_cols]))
    x_enc.fit(x_strings)

    # Make float columns into int columns
    float_cols = list(train_set.select_dtypes(include='float').columns)
    train_set[float_cols], test_set[float_cols] = train_set[float_cols].astype(int), test_set[float_cols].astype(int)

    if 'LATITUDE' in list(train_set.columns) and 'LONGITUDE' in list(train_set.columns):
        train_set[['LATITUDE', 'LONGITUDE']] = (train_set[['LATITUDE', 'LONGITUDE']].astype(int)+[90, 180])
        test_set[['LATITUDE', 'LONGITUDE']] = (test_set[['LATITUDE', 'LONGITUDE']].astype(int)+[90, 180])

    int_cols = list(train_set.select_dtypes(include='integer').columns)
    x_ints_min = pd.concat((train_set[int_cols], test_set[int_cols])).min().values
    x_ints_train = train_set[int_cols] - x_ints_min
    x_ints_test = test_set[int_cols] - x_ints_min

    X_train = np.concatenate((x_enc.transform(train_set[string_cols]),
                              x_ints_train.values), axis=1)
    X_test = np.concatenate((x_enc.transform(test_set[string_cols]),
                              x_ints_test.values), axis=1)



    return X_train, X_test, y_train_clf, y_test_clf, y_train_reg, y_test_reg, x_enc

def save_decoded_X(X, x_enc, cols, used, name):
    min_years = [used['START_YR'].astype(int).min(), 1978]
    X_inv = np.concatenate((x_enc.inverse_transform(X[:,:-4]), (X[:,-4:-2]+min_years).astype(int), X[:,-2:]), axis=1)
    pd.DataFrame(X_inv, columns=list(columns[:-2]+['Year']+columns[-2:])).to_csv(name+'.csv')

# Function to split rows into two DataFrames
def split_rows(group, test_fraction):

    num_rows = group.shape[0]
    if num_rows == 1:
        return None, None  # Exclude groups with only one sample

    num_sampled_rows = int(min(max(test_fraction * num_rows, 1), num_rows-1))  # At least one sample for each group

    test_df = group.sample(n=num_sampled_rows)
    train_df = group.drop(test_df.index)

    return train_df, test_df

In [5]:
def custom_interpolate(row):
    if row.count() >= 3:  # Check if there are enough values for polynomial interpolation
        return row.interpolate(method='polynomial', order=order, limit_direction='both')
    else:
        return row.interpolate(method='linear', limit_direction='both')

def metrics(y_true, y_pred, model_type='clf'):
    if model_type == 'clf':
        metric_dict = {'confusion': skm.confusion_matrix(y_true, y_pred),
                       'overall_acc': skm.accuracy_score(y_true, y_pred),
                       'average_acc': skm.balanced_accuracy_score(y_true, y_pred),
                       'kappa': skm.cohen_kappa_score(y_true, y_pred),
                       'IoU': skm.jaccard_score(y_true, y_pred, average='weighted')}
    elif model_type == 'reg':
        metric_dict = {'r2': skm.r2_score(y_true, y_pred),
                       'mae': skm.mean_absolute_error(y_true, y_pred),
                       'mse': skm.mean_squared_error(y_true, y_pred)}

    else: raise 'Incorrect model type'

    return metric_dict

In [6]:
## Define torch models and functions

class DeepNet(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(DeepNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out
# class ConvNet(nn.Module):
#     def __init__(self, input_channels, output_dim):
#         super(ConvNet, self).__init__()
#         self.conv1 = nn.Conv1d(input_channels, 16, kernel_size=2)
#         self.pool1 = nn.MaxPool1d(kernel_size=1)
#         self.conv2 = nn.Conv1d(16, 32, kernel_size=2)
#         self.pool2 = nn.MaxPool1d(kernel_size=1)
#         self.fc1 = nn.Linear(32 * 2, 256)
#         self.relu1 = nn.ReLU()
#         self.fc2 = nn.Linear(256, output_dim)
#
#     def forward(self, x):
#         batch_size = x.size(0)
#         out = self.conv1(x)
#         out = self.pool1(out)
#         out = self.conv2(out)
#         #out = self.pool2(out)
#         out = out.view(batch_size, -1)
#         out = self.fc1(out)
#         out = self.relu1(out)
#         out = self.fc2(out)
#         return out
#
#     def _calculate_fc_input_size(self):
#         # Calculate the input size for the fully connected layer
#         dummy_input = torch.zeros(1, 1, 9)
#         out = self.conv1(dummy_input)
#         out = self.pool1(out)
#         # out = self.conv2(out)
#         # out = self.pool2(out)
#         return out.size(2) * 32


def train(model, x_train, y_train, optimizer, criterion=torch.nn.CrossEntropyLoss(), epochs=100, verbose=True):
    """Training function for pytorch models"""
    model.train()

    for epoch in range(epochs):
        optimizer.zero_grad()  # Clear gradients
        y_pred = model(x_train) # Forward pass
        loss = criterion(y_pred, y_train) # Compute loss
        if verbose:
            print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
        loss.backward()  # Derive gradients
        optimizer.step()  # Update parameters based on gradients

    return loss, model, optimizer


def test(model, x_test, categorical=True):
    """Test function for pytorch models"""
    model.eval()
    y_pred = model(x_test) # Forward pass

    if categorical: # Get category with the highest probability
        _, y_pred_cats = torch.max(y_pred, dim = 1)
        return y_pred, y_pred_cats
    else: # Return raw prediction
        return y_pred

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMNet, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)

        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm1d(out_channels)
            )
        else:
            self.downsample = None

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, input_channels, output_dim, num_blocks=4):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(16)
        self.relu = nn.ReLU()
        self.residual_blocks = self._make_residual_blocks(16, num_blocks)
        self.fc = nn.Linear(16, output_dim)

    def _make_residual_blocks(self, channels, num_blocks):
        layers = []
        for i in range(num_blocks):
            layers.append(ResidualBlock(channels, channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.residual_blocks(out)
        out = torch.mean(out, dim=2)
        out = self.fc(out)
        return out


# class TransformerBlock(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_heads, dropout):
#         super(TransformerBlock, self).__init__()
#         self.attention = nn.MultiheadAttention(input_dim, num_heads, dropout=dropout)
#         self.dropout1 = nn.Dropout(dropout)
#         self.norm1 = nn.LayerNorm(input_dim)
#         self.fc = nn.Linear(input_dim, hidden_dim)
#         self.relu = nn.ReLU()
#         self.dropout2 = nn.Dropout(dropout)
#         self.norm2 = nn.LayerNorm(hidden_dim)
#
#     def forward(self, x):
#         out, _ = self.attention(x, x, x)
#         out = self.dropout1(out)
#         out = self.norm1(x + out)
#         out = self.fc(out)
#         out = self.relu(out)
#         out = self.dropout2(out)
#         out = self.norm2(out)
#         return out
#
#
# class TransformerNet(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_heads, num_blocks, output_dim, dropout):
#         super(TransformerNet, self).__init__()
#         self.transformer_blocks = self._make_transformer_blocks(input_dim, hidden_dim, num_heads, num_blocks, dropout)
#         self.fc = nn.Linear(hidden_dim, output_dim)
#
#     def _make_transformer_blocks(self, input_dim, hidden_dim, num_heads, num_blocks, dropout):
#         layers = []
#         for _ in range(num_blocks):
#             layers.append(TransformerBlock(input_dim, hidden_dim, num_heads, dropout))
#         return nn.Sequential(*layers)
#
#     def forward(self, x):
#         out = self.transformer_blocks(x)
#         out = torch.mean(out, dim=1)
#         out = self.fc(out)
#         return out

from torch.nn import TransformerEncoder, TransformerEncoderLayer

class TransformerNet(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256, num_layers=4, num_heads=8, dropout=0.1):
        super(TransformerNet, self).__init__()

        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.pos_encoding = PositionalEncoding(hidden_dim)
        encoder_layer = TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        x = self.transformer_encoder(x)
        x = torch.mean(x, dim=1)  # Average pooling over sequence length
        x = self.fc(x)
        return x

class PositionalEncoding(nn.Module):
    def __init__(self, hidden_dim, max_seq_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_seq_len, hidden_dim)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_dim, 2).float() * (-math.log(10000.0) / hidden_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

import torch
import torch.nn as nn

class TransformerNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2, num_heads=2, dropout=0.2):
        super(TransformerNet, self).__init__()

        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads),
            num_layers=num_layers
        )
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(0)  # Add a batch dimension
        x = x.permute(1, 0, 2)  # Permute dimensions for transformer input
        x = self.transformer_encoder(x)
        x = x.mean(dim=0)  # Average pooling
        x = self.fc(x)
        return x

In [7]:
def balance_classes_pt(X_train, X_test, y_train, y_test, col_name = 'Emissions'):
    y_train_pd = pd.Series(y_train, name=col_name)
    min_count = y_train_pd.reset_index().groupby(col_name).count().min()
    y_train_df = y_train_pd.reset_index().groupby(col_name).sample(min_count.values)
    y_train = y_train_df[col_name].values
    X_train = X_train[y_train_df.index]

    y_test_pd = pd.Series(y_test, name=col_name)
    min_count = y_test_pd.reset_index().groupby(col_name).count().min()
    y_test_df = y_test_pd.reset_index().groupby(col_name).sample(min_count.values)
    y_test = y_test_df[col_name].values
    X_test = X_test[y_test_df.index]

    return X_train, X_test, torch.tensor(y_train), torch.tensor(y_test)

In [9]:
### Training loop

# Define the hyperparameters to tune


# Input data
#data_path = "C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\data\classification_inputs/"
for input_data in ['petrochemicals']:
    print(input_data)
     # Output data
    #data_path = 'C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\models/datasets/'
    model_path = "/content/drive/MyDrive/PhD - Climate Change Mitigation/Outputs/"+input_data+'/'
    #'C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\models/'+input_data+'/' # '/content/models/'
    balance=False

    learning_rates = [0.001, 0.01, 0.1]
    hidden_sizes = [64, 128, 256]
    gap_filling_levels = [1,2,3]

    # Define divider for level 3
    if input_data == 'CT_manufacturing':
        divider = 'iso3_country'
        inference_cols = ['iso3_country', 'original_inventory_sector', 'asset_type']
        time_col = 'Timestep'
        timesteps = [str(i) for i in range(0,90)]
        graph_cols, max_edges = [0,1,3,5], 100

    elif input_data == 'petrochemicals':
        divider = 'COUNTRY/TERRITORY'
        inference_cols = ['PRODUCT', 'COUNTRY/TERRITORY']
        time_col = 'Year'
        timesteps = [str(i) for i in range(1978,2051)]
        graph_cols, max_edges = [0,3], 30
        balance = True

    elif input_data == 'unfccc':
        divider='Party'
        inference_cols = ['Party', 'Category']
        time_col = 'Year'
        timesteps = [str(i) for i in range(1990,2021)]
        graph_cols, max_edges = [0], 100

    ## Parameters
    max_test_set = 100000
    random_state = 2
    test_size = 0.3
    regression = False

    # Gap level
    for gap_filling_level in gap_filling_levels:
        print('Gap level '+str(gap_filling_level))
        X_train_pt = torch.load(data_path+'X_train_pt-'+input_data+'-'+str(gap_filling_level)+'.pt')
        X_test_pt = torch.load(data_path+'X_test_pt-'+input_data+'-'+str(gap_filling_level)+'.pt')
        y_train_pt = torch.load(data_path+'y_train_pt-'+input_data+'-'+str(gap_filling_level)+'.pt')
        y_test_pt = torch.load(data_path+'y_test_pt-'+input_data+'-'+str(gap_filling_level)+'.pt')


        if balance:
          X_train_pt, X_test_pt, y_train_pt, y_test_pt = balance_classes_pt(X_train_pt, X_test_pt, y_train_pt, y_test_pt)

        X_train_pt, X_test_pt, y_train_pt, y_test_pt = X_train_pt.to('cuda:0'), X_test_pt.to('cuda:0'), y_train_pt.to('cuda:0'), y_test_pt.to('cuda:0')

        input_dim = X_train_pt.shape[1]
        input_channels = 1#X_train_pt.shape[1]
        output_dim = 4
        hidden_dim1 = 64
        hidden_dim2 = 64


        models = [
                    ("DeepNet", DeepNet(input_dim, hidden_dim1, hidden_dim2, output_dim).to('cuda')),
                    ('LSTM', LSTMNet(input_dim, hidden_dim1, output_dim).to('cuda')),
                    ('ResNet', ResNet(input_channels, output_dim).to('cuda')),
                    #('Transformer', TransformerNet(input_dim,256, output_dim))
                ]

        # Define a list of models to train

        best_accuracy = {}
        best_learning_rate = {}
        best_hidden_size = {}

        for model_name, model in models:

            best_accuracy[model_name] = 0
            best_learning_rate[model_name] = 0
            best_hidden_size[model_name] = 0

            print(f"Training {model_name}")
            for learning_rate in learning_rates:
                print(learning_rate)
                for hidden_size in hidden_sizes:
                    print(hidden_size)
                    model_type = 'reg' if regression else 'clf'
                    model_file = model_path + model_type + '_' + model_name + '_l' + str(str(gap_filling_level)) + '_' + date.today().strftime("%y%m%d")

                    # Set the hyperparameters
                    if model_name == 'DeepNet':
                        model.hidden_size1 = hidden_size
                        model.hidden_size2 = hidden_size
                    if model_name == 'LSTM':
                        model.hidden_dim1 = hidden_size
                    # Reshape the input to have 4 dimensions
                    if model_name == 'LSTM' or model_name=='ResNet':
                        # Reshape the input data to have 3 channels
                        X_train_in = X_train_pt.unsqueeze(1).to('cuda:0')
                        X_test_in = X_test_pt.unsqueeze(1).to('cuda:0')
                    else:
                        X_train_in = X_train_pt.to('cuda:0')
                        X_test_in = X_test_pt.to('cuda:0')


                    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
                    criterion = nn.CrossEntropyLoss()
                    epochs = 100

                    loss, model, optimizer = train(model, X_train_in, y_train_pt, optimizer, criterion, epochs=epochs)

                    #torch.save(model.state_dict(), model_file+'.pt')

                    if regression:
                        y_pred = test(model, X_test_in)
                    else:
                        _, y_pred = test(model, X_test_in)

                    scores = metrics(y_test_pt.cpu().numpy(), y_pred.cpu().numpy(), model_type)

                    np.save(model_file+'_'+str(learning_rate)+'_'+str(hidden_size)+'_100iter.npy', scores)

        #             # Check if the current combination of hyperparameters is the best
                    if scores['average_acc'] > best_accuracy[model_name]:
                        torch.save(model.state_dict(), model_file+'_'+str(learning_rate)+'_'+str(hidden_size)+'.pt')
                        # best_accuracy[model_name] = scores['average_acc']
                        # best_learning_rate[model_name] = learning_rate
                        # best_hidden_size[model_name] = hidden_size

        # # Train the best models with the best hyperparameters for 100 epochs
        # for model_name, model in models:
        #     best_lr = best_learning_rate[model_name]
        #     best_hs = best_hidden_size[model_name]
        #     model_type = 'reg' if regression else 'clf'
        #     model_file = model_path + model_type + '_' + model_name + '_l' + str(str(gap_filling_level)) + '_' + date.today().strftime("%y%m%d")

        #     # Set the hyperparameters
        #     if model_name == 'DeepNet':
        #         model.hidden_size1 = best_hs
        #         model.hidden_size2 = best_hs
        #     if model_name == 'LSTM':
        #         model.hidden_dim1 = best_hs
        #     # Reshape the input to have 4 dimensions
        #     if model_name == 'LSTM' or model_name=='ResNet':
        #         # Reshape the input data to have 3 channels
        #         X_train_in = X_train_pt.unsqueeze(1).to('cuda:0')
        #         X_test_in = X_test_pt.unsqueeze(1).to('cuda:0')
        #     else:
        #         X_train_in = X_train_pt.to('cuda:0')
        #         X_test_in = X_test_pt.to('cuda:0')


        #     optimizer = torch.optim.Adam(model.parameters(), lr=best_lr)
        #     criterion = nn.CrossEntropyLoss()
        #     epochs = 100

        #     #model = model
        #     loss, model, optimizer = train(model, X_train_in, y_train_pt, optimizer, criterion, epochs=epochs)

        #     torch.save(model.state_dict(), model_file+'.pt')

        #     if regression:
        #         y_pred = test(model, X_test_in)
        #     else:
        #         _, y_pred = test(model, X_test_in)

        #     scores = metrics(y_test_pt, y_pred, model_type)

        #     np.save(model_file+'_'+str(best_lr)+'_'+str(best_hs)+'_100iter.npy', scores)

petrochemicals
Gap level 2
Training ResNet
0.1
128
Epoch 0: train loss: 1.7057759761810303
Epoch 1: train loss: 2.1262927055358887
Epoch 2: train loss: 2.043576717376709
Epoch 3: train loss: 1.6805980205535889
Epoch 4: train loss: 1.5312809944152832
Epoch 5: train loss: 1.4587137699127197
Epoch 6: train loss: 1.4209082126617432
Epoch 7: train loss: 1.4029253721237183
Epoch 8: train loss: 1.3970422744750977
Epoch 9: train loss: 1.3973758220672607
Epoch 10: train loss: 1.400516390800476
Epoch 11: train loss: 1.4033150672912598
Epoch 12: train loss: 1.4038201570510864
Epoch 13: train loss: 1.4009045362472534
Epoch 14: train loss: 1.394986867904663
Epoch 15: train loss: 1.3881986141204834
Epoch 16: train loss: 1.3820364475250244
Epoch 17: train loss: 1.380053997039795
Epoch 18: train loss: 1.380537509918213
Epoch 19: train loss: 1.3770114183425903
Epoch 20: train loss: 1.3724151849746704
Epoch 21: train loss: 1.3703359365463257
Epoch 22: train loss: 1.3703573942184448
Epoch 23: train loss:

In [ ]:
torch.cuda.empty_cache()